In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import json
import logging
import pickle
import time
import datetime

In [2]:
conf = SparkConf().setAppName("test").setMaster("local[*]")

In [3]:
scc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/14 16:01:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark = SparkSession(scc)

In [5]:
schema = pickle.load(open("schema.pkl", 'rb'))

In [6]:
#query 1 .alias("language")

import datetime as dt
df = spark.read.schema(schema).option("multiline","true").json("data.json")
x = df.selectExpr("CAST(birth AS DATE)","CAST(death AS DATE)")
y = x.withColumn('death', F.when(F.col('death').isNull(), dt.datetime.now().date()).otherwise(F.col('death')))
z= y.withColumn("Age", F.year(F.col("death"))-F.year(F.col("birth")))
z.show()

+----------+----------+---+
|     birth|     death|Age|
+----------+----------+---+
|1924-12-03|2007-03-17| 83|
|1927-09-04|2011-12-24| 84|
|1906-12-09|1992-01-01| 86|
|1926-08-27|2002-08-10| 76|
|1931-10-12|2002-06-29| 71|
|1956-01-31|2022-12-14| 66|
|1941-09-09|2011-10-12| 70|
|1965-04-14|2022-12-14| 57|
|1955-05-19|2022-12-14| 67|
|1955-05-19|2022-12-14| 67|
+----------+----------+---+



In [7]:
# query 2

df.select(F.size(F.col("contribs")).alias("num_contribs")).show()

+------------+
|num_contribs|
+------------+
|           4|
|           3|
|           4|
|           2|
|           2|
|           1|
|           2|
|           1|
|           1|
|           1|
+------------+



In [8]:
# query 3
from pyspark.sql.types import ArrayType,StringType

def min_max(awards_year):
    mina = 10000
    maxa = 0
    # temp = []
    if awards_year is None:
        return []
    for year in awards_year:
        # temp.append(int(year))
        if int(year)>maxa:
            maxa = int(year)
            
        if int(year)<mina:
            mina = int(year)
    # ,[min(temp),max(temp)]
    return [mina,maxa]


convertUDF = F.udf(lambda z: min_max(z))
df.select(convertUDF(F.col("awards.year")).alias("min_max") ).show(truncate=False)

+------------+
|min_max     |
+------------+
|[1967, 1993]|
|[1971, 1990]|
|[1969, 1991]|
|[1999, 2001]|
|[1999, 2001]|
|[2001, 2003]|
|[1983, 2011]|
|[2011, 2011]|
|[2002, 2007]|
|[]          |
+------------+



In [9]:
df = df.selectExpr("name","contribs","awards","CAST(birth AS DATE)","CAST(death AS DATE)")
df = df.withColumn('death', F.when(F.col('death').isNull(), dt.datetime.now().date()).otherwise(F.col('death')))
df= df.withColumn("age", F.year(F.col("death"))-F.year(F.col("birth")))
final = df.select(F.col("name"), F.col("age"),F.size(F.col("contribs")).alias("num_contribs"), convertUDF(F.col("awards.year")).alias("min_max"))
final.show()


+--------------------+---+------------+------------+
|                name|age|num_contribs|     min_max|
+--------------------+---+------------+------------+
|{null, John, Backus}| 83|           4|[1967, 1993]|
|{null, John, McCa...| 84|           3|[1971, 1990]|
|{null, Grace, Hop...| 86|           4|[1969, 1991]|
|{null, Kristen, N...| 76|           2|[1999, 2001]|
|{null, Ole-Johan,...| 71|           2|[1999, 2001]|
|{null, Guido, van...| 66|           1|[2001, 2003]|
|{null, Dennis, Ri...| 70|           2|[1983, 2011]|
|{Matz, Yukihiro, ...| 57|           1|[2011, 2011]|
|{null, James, Gos...| 67|           1|[2002, 2007]|
|{null, Martin, Od...| 67|           1|          []|
+--------------------+---+------------+------------+



In [12]:
final.write.mode("append").parquet("new2.parquet")

In [13]:
df = spark.read.parquet("new2.parquet")
df.show()

+--------------------+---+------------+------------+
|                name|age|num_contribs|     min_max|
+--------------------+---+------------+------------+
|{null, John, Backus}| 83|           4|[1967, 1993]|
|{null, John, McCa...| 84|           3|[1971, 1990]|
|{null, Grace, Hop...| 86|           4|[1969, 1991]|
|{null, Kristen, N...| 76|           2|[1999, 2001]|
|{null, Ole-Johan,...| 71|           2|[1999, 2001]|
|{null, Guido, van...| 66|           1|[2001, 2003]|
|{null, Dennis, Ri...| 70|           2|[1983, 2011]|
|{Matz, Yukihiro, ...| 57|           1|[2011, 2011]|
|{null, James, Gos...| 67|           1|[2002, 2007]|
|{null, Martin, Od...| 67|           1|          []|
+--------------------+---+------------+------------+

